# DL2DWH Lab

In this lab you are asked to cover the following points:
1. Data ingestion
2. Data understanding
3. Design and build a relational DWH
4. Query the DWH with Tableau (Desktop)
5. (Bonus) Plot the provenance graph

In [ ]:
!pip install awswrangler
!pip install pandas
!pip install sklearn
!pip install prov
!pip install seaborn
!pip install pydot
!pip install graphviz
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import awswrangler as wr
import seaborn as sns
import prov

## Data ingestion

- Upload the following files in S3 (remember the naming conventions of the data lake areas!)
    - [http://big.csr.unibo.it/projects/nosql-datasets/sales_fact_1997.slice-0-1.v0.0.1.csv](http://big.csr.unibo.it/projects/nosql-datasets/sales_fact_1997.slice-0-1.v0.0.1.csv)
    - [http://big.csr.unibo.it/projects/nosql-datasets/sales_fact_1997.slice-1-2.v0.0.1.csv](http://big.csr.unibo.it/projects/nosql-datasets/sales_fact_1997.slice-1-2.v0.0.1.csv)
    - [http://big.csr.unibo.it/projects/nosql-datasets/sales_fact_1997.slice-2-3.v0.0.1.csv](http://big.csr.unibo.it/projects/nosql-datasets/sales_fact_1997.slice-2-3.v0.0.1.csv)
- Merge/Append them into a single Pandas DataFrame
    - Which is the underlying assumption so that `append` works?
    - See [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.append.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.append.html)
- Save the processed dataset into S3

In [ ]:
file_names = ["s3://staging-clean-sales-911/sales_fact_1997.slice-0-1.v0.0.1.csv", "s3://staging-clean-sales-911/sales_fact_1997.slice-1-2.v0.0.1.csv", "s3://staging-clean-sales-911/sales_fact_1997.slice-2-3.v0.0.1.csv",]
    
df = pd.concat(map(pd.read_csv, ["s3://landing-raw-sales-911/sales_fact_1997.slice-0-1.v0.0.1.csv", 
                                "s3://landing-raw-sales-911/sales_fact_1997.slice-1-2.v0.0.1.csv", 
                                "s3://landing-raw-sales-911/sales_fact_1997.slice-2-3.v0.0.1.csv"]), ignore_index = True)
print(df)
    

In [10]:
landing_bucket = "s3://landing-raw-sales-911/"
staging_bucket = "s3://staging-clean-sales-911/"

In [ ]:
# wr.s3.to_csv(df=df1, path=output_bucket + "cleaned-v1-" + file_name, index=False)

wr.s3.to_csv(df = df, path = staging_bucket + "merged-sales-fact", index = False)

## Data understanding

- What is the dataset about?
- Which are candidate levels/measures?
- Check and fix the missing values (if any)
- How many products have been sold?
    - See [https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html)
- Which functional dependencies hold among the levels?
    - Given a relation R, a set of attributes X in R is said to functionally determine another set of attributes Y, also in R, (written X → Y) if, and only if, each X value is associated with precisely one Y value; R is then said to satisfy the functional dependency X → Y.
    - See [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html)
    - Hint: `df.groupby([...])` + `nunique()`
- Are there relationships between measures?
    - See [https://seaborn.pydata.org/generated/seaborn.pairplot.html](https://seaborn.pydata.org/generated/seaborn.pairplot.html)
    - See [https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)

In [ ]:
print(df.isnull().sum())

In [ ]:
print(df.columns)

In [ ]:
print(df.info())

## Considerations:

List of columns: ['C. City', 'C. Country', 'C. State', 'Customer', 'Yearly Income',
       'Category', 'Subcategory', 'Family', 'Product', 'S. City', 'S. Country',
       'S. State', 'Store', 'Type', 'Sales Date', 'Store Cost', 'Store Sales',
       'Unit Sales']
    
**FD**:
City -> State -> Country [to be checked]
Category -> Subcategory -> Family -> Product [to be checked]
    
**DT_Customer** = (C.City, C.Country, C.State, Customer, Yearly_Income)
**DT_Product** = (Category, Subcategory, Family, Product)
**DT_Store** = (S.City, S.Country, S.State, Store, Type)

**FT_Sale** = (Customer:DT_Customer, Product:DT_Product, Sales_Date, *Store_Cost*, *Store_Sales*, *Unit_Sales*)

*Measures*

## Design and build a relational DWH

- Create the database on Amazon RDS
- Design the relational schema of the DWH (which are the functional dependencies?)
- Upload the tables into rhw DWH

In [ ]:
raw_data = wr.s3.read_csv(staging_bucket + "merged-sales-fact")
raw_data

In [ ]:
print(raw_data.columns)

In [ ]:
dt_customer = raw_data[['C. City', 'C. Country', 'C. State', 'Customer', 'Yearly Income']].drop_duplicates()
dt_customer

In [ ]:
dt_product = raw_data[['Category', 'Subcategory', 'Family', 'Product']].drop_duplicates()
dt_product

In [ ]:
dt_store = raw_data[['S. City', 'S. Country',
       'S. State', 'Store', 'Type']].drop_duplicates()
dt_store

In [ ]:
dt_date = raw_data[['Sales Date']].drop_duplicates()
dt_date

In [ ]:
ft_sales = raw_data[['Sales Date', 'Store', 'Unit Sales', 'Product',
                     'Customer', 'Store Sales', 'Store Cost', 'Unit Sales']].drop_duplicates()
ft_sales

In [42]:
host = "database-lab.cepvkrc8ii6y.us-east-1.rds.amazonaws.com"
port = 5432
user = "postgres"
pwd = "bigdata2022"
db = "postgres"

from sqlalchemy import create_engine
s = 'postgresql://{}:{}@{}:{}/{}'.format(user, pwd, host, str(port), db)
engine = create_engine(s)

In [54]:
dt_customer.to_sql('customer', engine, index=False, if_exists='replace')
dt_product.to_sql('product', engine, index=False, if_exists='replace')
dt_store.to_sql('store', engine, index=False, if_exists='replace')
dt_date.to_sql('date', engine, index=False, if_exists='replace')

In [57]:
ft_sales.to_sql('sale', engine, index=False, if_exists='replace')

## Query the DWH with Tableau (Desktop)

- Import data from Amazon RDS into Tableau
- Join the tables
- Set `Sales Date` as a Datetime attribute both in the fact and dimension tables
- Build the dimensions and hierarchies in Tableau
- Complete the following exercises

### Exercise 1

1. Using a bar chart, display the `sum` of `Store Sales` for each `S. State`
     - Which state has the highest sales?
2. Drill down to view sales at the `S. City` level.
     - Are there cities with much lower sales than others?
3. How many stores (`Stores`) are there in each `S. State`? What about in each `S. City`?
     - Tip: Use `COUNTD` aggregation. Why?
4. Can low sales in some cities be imputed to the number of stores?
     - Display sales at the `S. City` level (as done above) and associate the color property with the number of distinct stores (`COUNTD(store)`)

### Exercise 2

5. Given the last visualization, associate the `S. Type` field with the color property.
    - What interesting pattern can you see?
6. Display the sales (`SUM`) for each `S. Type`.
    - What discrepancy is possible to notice with respect to the previous chart?
7. Associate the number of stores (`Store`) with the color property and the label property
    - What caused the discrepancy between the previous two visualizations?

### Exercise 3

8. Using a line chart, visualize the monthly sales trend
    - What pattern is present?
9. Divide the previous chart by `S. State` (one axis for each state)
    - Is the previous pattern present in each `State`?
    - Tip: by default the axes all have the same range: on any axis, right-click > `Edit Axis` > `Select Independent axis...`
10. Given the previous chart, visualize how much impact the various `Family`s have on total sales while maintaining the monthly trend visualization
    - What might be a good visualization?
    - Tip: associate each Family with a mark property and possibly change the mark type
    - Tip: you can change the mark type from the drop-down menu in the mark panel

## (Bonus) Plot the provenance graph

- Plot the provenance graph representing the previous transformations using the PROV standard

In [6]:
from prov.model import ProvDocument
from prov.dot import prov_to_dot
from IPython.display import Image

def plot():
    # Create a new provenance document
    d1 = ProvDocument()  # d1 is now an empty provenance document
    
    d1.add_namespace('s3', 'https://s3.console.aws.amazon.com/s3')
    d1.add_namespace('rds', 'https://us-east-1.console.aws.amazon.com/rds')
    d1.add_namespace('unibo', 'https://www.unibo.it')
    d1.add_namespace('pd', 'https://pandas.pydata.org/')
    
    # TODO

    # visualize the graph
    dot = prov_to_dot(d1)
    dot.write_png('prov.png')
    return Image('prov.png')

In [7]:
plot()